# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import os
import shutil

import azureml.core
from azureml.widgets import RunDetails
from azureml.core.workspace import Workspace
from azureml.core.experiment import Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.48.0


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

The external data is going to be loaded in train.py script. Here's a snapshot of the script:

```
# Create TabularDataset using TabularDatasetFactory
url_path = "https://raw.githubusercontent.com/ratheswaran/directory_name/main/train.csv"
dataset = TabularDatasetFactory.from_delimited_files(path=url_path)
data_df = dataset.to_pandas_dataframe()
```

In [2]:
ws = Workspace.from_config()
experiment_name = 'udacity-capstone-project-hyperdrive'

experiment=Experiment(ws, experiment_name)

## Initialize Workspace

In [3]:
ws = Workspace.from_config()

print('Workspace name: ' + ws.name,
      'Azure region: ' + ws.location,
      'Subscription id: ' + ws.subscription_id,
      'Resource group: ' + ws.resource_group, sep='\n')

Workspace name: quick-starts-ws-228138
Azure region: southcentralus
Subscription id: 976ee174-3882-4721-b90a-b5fef6b72f24
Resource group: aml-quickstarts-228138


In [5]:
run = experiment.start_logging()

## Attach Cluster

In [6]:
# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "aml-compute"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

Found existing cluster, use it.
Succeeded....................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

The pipeline we are using here consists of a custom-coded Scikit-learn model logistic regression model stored in train.py script and a Hyperdrive run sweeping over model paramters. The following steps are part of the pipeline:

Data preprocessing
Splitting data into train and test sets
Setting logistic regression parameters:
--C - Inverse of regularization strenght
--max_iter - Maximum number of iterations convergence
`RandomParameterSampling` defines random sampling over a hyperparameter search space. In this sampling algorithm, parameter values are chosen from a set of discrete values or a distribution over a continuous range. This has an advantage against GridSearch method that runs all combinations of parameters and requires large amount of time to run.

For the Inverse of regularization strenght parameter I have chosen uniform distribution with min=0.0001 and max=1 For the Maximum number of iterations convergence I inputed a range of values (5, 25, 50, 100, 200, 500, 1000)

`BanditPolicy` Class Defines an early termination policy based on slack criteria, and a frequency and delay interval for evaluation. This greatly helps to ensure if model with given parameters is not performing well, it is turned down instead of running it for any longer.

In [13]:
import azureml.core
from azureml.widgets import RunDetails
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        "--C": uniform(0.0001, 1.0),
        "--max_iter": choice(5, 25, 50, 100, 200, 500, 1000)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='myenv.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job

src = ScriptRunConfig(source_directory=".",
                      script='train.py',
                      compute_target=compute_target,
                      environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                    hyperparameter_sampling=ps,
                    policy=policy,
                    primary_metric_name='Accuracy',
                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                    max_total_runs=16,
                    max_concurrent_runs=4)

In [14]:
#TODO: Submit your experiment
hyperdrive_run = experiment.submit(config=hyperdrive_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [15]:
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_1ba84d5a-e2d8-4e69-9c49-b8596c52b7b9
Web View: https://ml.azure.com/runs/HD_1ba84d5a-e2d8-4e69-9c49-b8596c52b7b9?wsid=/subscriptions/976ee174-3882-4721-b90a-b5fef6b72f24/resourcegroups/aml-quickstarts-228138/workspaces/quick-starts-ws-228138&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2023-03-18T09:41:34.019083][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2023-03-18T09:41:34.4741772Z][SCHEDULER][INFO]Scheduling job, id='HD_1ba84d5a-e2d8-4e69-9c49-b8596c52b7b9_0' 
[2023-03-18T09:41:34.5836751Z][SCHEDULER][INFO]Scheduling job, id='HD_1ba84d5a-e2d8-4e69-9c49-b8596c52b7b9_1' 
[2023-03-18T09:41:34.6239734Z][SCHEDULER][INFO]Scheduling job, id='HD_1ba84d5a-e2d8-4e69-9c49-b8596c52b7b9_2' 
[2023-03-18T09:41:34.716745][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.
[2023-03-18T09:41:34.7769933Z][SCHEDULER][INFO]Scheduling job, id='HD_1ba84d5a-e2d8-4e69-9c49-b8596c52b7

{'runId': 'HD_1ba84d5a-e2d8-4e69-9c49-b8596c52b7b9',
 'target': 'aml-compute',
 'status': 'Completed',
 'startTimeUtc': '2023-03-18T09:41:32.972554Z',
 'endTimeUtc': '2023-03-18T09:50:40.636658Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '530541d2-5c0e-4651-b52a-53d6ba782ff6',
  'user_agent': 'python/3.8.10 (Linux-5.15.0-1031-azure-x86_64-with-glibc2.17) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.48.0',
  'space_size': 'infinite_space_size',
  'score': '0.8156424581005587',
  'best_child_run_id': 'HD_1ba84d5a-e2d8-4e69-9c49-b8596c52b7b9_10',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_1ba84d5a-e2d8-4e69-9c49-b8596c52b7b9_10'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryV

KeyError: 'log_files'

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [16]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print("Best Experiment Run:")
print(f" Best Run Id: {best_run.id}")
print(f" Accuracy: {best_run_metrics['Accuracy']}")
print(f" Regularization Strength: {best_run_metrics['Regularization Strength:']}")
print(f" Max iterations: {best_run_metrics['Max iterations:']}")

Best Experiment Run:
 Best Run Id: HD_1ba84d5a-e2d8-4e69-9c49-b8596c52b7b9_10
 Accuracy: 0.8156424581005587
 Regularization Strength: 0.9120482384941757
 Max iterations: 25


In [17]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-capstone-project-hyperdrive,HD_1ba84d5a-e2d8-4e69-9c49-b8596c52b7b9_10,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [18]:
#TODO: Save the best model
best_model = best_run.register_model(model_name='hyperdrive_model.pkl',
                                     model_path='./outputs/')

best_model.download(target_dir='', exist_ok=True)

'outputs/model_hyppar_tunning.joblib'

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

